In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [48]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 08:00:00+00:00


In [49]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [50]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-05 02:18:38,092 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:18:38,099 INFO: Initializing external client
2025-03-05 02:18:38,099 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:18:38,620 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695


In [51]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.70s) 


In [52]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,230,215.0,2025-03-05 01:00:00+00:00
1,42,7.0,2025-03-05 01:00:00+00:00
2,99,0.0,2025-03-05 01:00:00+00:00
3,235,0.0,2025-03-05 01:00:00+00:00
4,63,0.0,2025-03-05 01:00:00+00:00
...,...,...,...
1752,163,238.0,2025-03-05 07:00:00+00:00
1753,213,0.0,2025-03-05 07:00:00+00:00
1754,261,24.0,2025-03-05 07:00:00+00:00
1755,48,29.0,2025-03-05 07:00:00+00:00


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  1757 non-null   int32                  
 1   predicted_demand    1757 non-null   float64                
 2   pickup_hour         1757 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 34.4 KB


In [54]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-05 02:18:51,795 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:18:51,799 INFO: Initializing external client
2025-03-05 02:18:51,799 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:18:52,299 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Current UTC time: 2025-03-05 07:18:51.795042+00:00
Next hour: 2025-03-05 08:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 08:00
Found 0 records


In [55]:
now = datetime.now(timezone.utc)

In [56]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  1757 non-null   int32                  
 1   predicted_demand    1757 non-null   float64                
 2   pickup_hour         1757 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 34.4 KB


In [58]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [59]:
dt = current_date.ceil('h')


In [60]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-05 02:19:32,234 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:19:32,239 INFO: Initializing external client
2025-03-05 02:19:32,239 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:19:32,765 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.54s) 


In [61]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.66s) 


,pickup_location_id,predicted_demand,pickup_hour


In [62]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,230,215.0,2025-03-05 01:00:00+00:00
1,42,7.0,2025-03-05 01:00:00+00:00
2,99,0.0,2025-03-05 01:00:00+00:00
3,235,0.0,2025-03-05 01:00:00+00:00
4,63,0.0,2025-03-05 01:00:00+00:00
...,...,...,...
1752,163,238.0,2025-03-05 07:00:00+00:00
1753,213,0.0,2025-03-05 07:00:00+00:00
1754,261,24.0,2025-03-05 07:00:00+00:00
1755,48,29.0,2025-03-05 07:00:00+00:00


In [63]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [64]:
predictions = fetch_next_hour_predictions()

2025-03-05 02:19:58,318 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:19:58,322 INFO: Initializing external client
2025-03-05 02:19:58,322 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:19:58,854 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Current UTC time: 2025-03-05 07:19:58.318006+00:00
Next hour: 2025-03-05 08:00:00+00:00
Found 0 records


In [65]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-05 02:20:00,845 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:20:00,850 INFO: Initializing external client
2025-03-05 02:20:00,850 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:20:01,345 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695


In [66]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.63s) 


In [67]:
df["pickup_hour"].max()

Timestamp('2025-03-05 07:00:00+0000', tz='Etc/UTC')

In [68]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 08:00:00+00:00


In [69]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
